# Clasificación de canciones con máquinas de soporte vectorial
## Javier Andres Tellez Ortiz 201617861

### Se descarga el conjunto de datos y se descomprime

In [1]:
import wget
from zipfile import ZipFile

##Se descarga el archivo del repositorio 
file = wget.download("http://millionsongdataset.com/sites/default/files/AdditionalFiles/msd_genre_dataset.zip")

##Se abre el archivo y se descomprime
zpFile = ZipFile(file)
zpFile.extractall()
zpFile.close()

100% [........................................................................] 12656044 / 12656044

In [2]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("msd_genre_dataset.txt", skiprows = range(9))

dataset.dropna()
dataset.head()

,%genre,track_id,artist_name,title,loudness,tempo,time_signature,key,mode,duration,...,var_timbre3,var_timbre4,var_timbre5,var_timbre6,var_timbre7,var_timbre8,var_timbre9,var_timbre10,var_timbre11,var_timbre12
0,classic pop and rock,TRFCOOU128F427AEC0,Blue Oyster Cult,Mes Dames Sarat,-8.697,155.007,1,9,1,246.33424,...,1255.514569,580.030472,598.485223,575.337671,322.068603,321.726029,232.700609,186.805303,181.938688,151.508011
1,classic pop and rock,TRNJTPB128F427AE9F,Blue Oyster Cult,Screams,-10.659,148.462,1,4,0,189.80526,...,2007.653070,1043.474073,585.694981,564.013736,510.177022,400.200186,365.119588,238.099708,197.933757,251.577525
2,classic pop and rock,TRLFJHA128F427AEEA,Blue Oyster Cult,Dance The Night Away,-13.494,112.909,1,10,0,158.19710,...,1204.856777,2736.520024,730.233239,665.203452,535.775111,439.335059,486.822970,265.333860,447.097987,251.880724
3,classic pop and rock,TRCQZAG128F427DB97,Blue Oyster Cult,Debbie Denise,-12.786,117.429,4,7,1,250.22649,...,809.755802,563.908070,492.803819,378.382799,372.875044,231.941957,246.313305,168.400152,85.282462,339.897173
4,classic pop and rock,TRNXMNM128F427DB8C,Blue Oyster Cult,(Don't Fear) The Reaper,-14.093,141.536,4,9,0,307.06893,...,1093.684935,343.556047,889.163314,218.111796,304.862864,178.352161,440.478867,142.669283,81.061326,208.355152


In [3]:
dataset = dataset.drop(columns = ["track_id", "artist_name", "title"])
features = dataset.columns.tolist()
dataset = dataset[(dataset["%genre"] == "jazz and blues") | (dataset["%genre"] == "soul and reggae")]

### Se obtienen los datos de los géneros de interés y se muestra la cantidad de cada clase

In [4]:
pd.DataFrame(dataset["%genre"].value_counts())

,%genre
jazz and blues,4334
soul and reggae,4016


### Se divide la información del archivo entre etiquetas y carácteristicas

In [5]:
y = dataset["%genre"].values
X = dataset.values[:,1:]

### Como se evidenció en entregas anteriores, es necesaria la estándarizacion de los datos. Además se tranforman las etiquetas de cada calse en números

In [6]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

scaler = StandardScaler()
lblEncoder = LabelEncoder()

y = lblEncoder.fit(np.unique(y)).transform(y)
##Se estandarizan los datos provenientes del archivo
X = scaler.fit(X).transform(X)

### Se dividen los datos en datos de entrenamiento y datos de prueba, tomando el 20% del total para este último fin. Antes de realizar la división, los datos son barajados

In [7]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(X, y, test_size = 0.2, random_state = 7861)

genres, cantidad = np.unique(y_test, return_counts = True)
total_test_data = sum(cantidad)
labels = lblEncoder.inverse_transform(genres)
print("Datos de prueba clase %s (%s): %d" % (genres[0],labels[0],cantidad[0]))
print("Datos de prueba clase %s (%s): %d" % (genres[1],labels[1],cantidad[1]))
print("Total datos prueba: %d" % total_test_data)

Datos de prueba clase 0 (jazz and blues): 879
Datos de prueba clase 1 (soul and reggae): 791
Total datos prueba: 1670


### Se toma el 10% de los datos de entrenamiento como datos de validación, esto con el objetivo de comparar cada uno de los modelos que serán entrenados

In [8]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.1, random_state = 7861)

genres, cantidad = np.unique(y_train, return_counts = True)
labels = lblEncoder.inverse_transform(genres)
print("Datos de entrenamiento clase %s (%s): %d" % (genres[0],labels[0],cantidad[0]))
print("Datos de entrenamiento clase %s (%s): %d" % (genres[1],labels[1],cantidad[1]))

genres, cantidad = np.unique(y_validation, return_counts = True)
labels = lblEncoder.inverse_transform(genres)
print("Datos de validacion clase %s (%s): %d" % (genres[0],labels[0],cantidad[0]))
print("Datos de validacion clase %s (%s): %d" % (genres[1],labels[1],cantidad[1]))

Datos de entrenamiento clase 0 (jazz and blues): 3123
Datos de entrenamiento clase 1 (soul and reggae): 2889
Datos de validacion clase 0 (jazz and blues): 332
Datos de validacion clase 1 (soul and reggae): 336


### Se define los distintos anchos del kernel gaussiano a usar en la máquina de soporte vectorial y algunos valores para la constante C

In [9]:
C = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]
sigma = [0.00001, 0.00003,0.0001,0.0003,0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3]

### Para cada ancho de kernel considerado, se entrena un SVM con las distintas constantes C escogidas. Se evalúa cada modelo en los datos de validación y se almacenan la exactitud obtenida en una matriz para su posterior análisis

In [24]:
from sklearn.svm import SVC

score_matrix = np.zeros((len(sigma), len(C)))
i = 0
j = 0

for std in sigma:
    for regularization in C:
        
        machine = SVC(C=regularization, 
                      kernel='rbf',
                      gamma=std)
        
        machine.fit(X_train, y_train)
        score = machine.score(X_validation, y_validation)
        score_matrix[i,j] = score
        j += 1
    
    j = 0
    i += 1

### Se genera una tabla para observar los resultados alcanzados y obtener los parámetros para el mejor modelo 

In [25]:
model_results = pd.DataFrame(score_matrix)
model_results['Sigma'] = sigma
model_results = model_results.set_index('Sigma')
model_results.columns = C
model_results.index.name = 'Sigma \ C'
model_results

,0.001,0.003,0.010,0.030,0.100,0.300,1.000,3.000,10.000,30.000,100.000
Sigma \ C,,,,,,,,,,,
0.00001,0.497006,0.497006,0.497006,0.497006,0.497006,0.497006,0.531437,0.808383,0.821856,0.820359,0.835329
0.00003,0.497006,0.497006,0.497006,0.497006,0.497006,0.516467,0.808383,0.817365,0.820359,0.832335,0.839820
0.00010,0.497006,0.497006,0.497006,0.497006,0.528443,0.809880,0.820359,0.821856,0.833832,0.841317,0.841317
0.00030,0.497006,0.497006,0.497006,0.511976,0.806886,0.817365,0.821856,0.833832,0.841317,0.848802,0.851796
0.00100,0.497006,0.497006,0.520958,0.805389,0.818862,0.821856,0.833832,0.842814,0.850299,0.850299,0.854790
0.00300,0.497006,0.497006,0.794910,0.817365,0.824850,0.827844,0.844311,0.853293,0.857784,0.862275,0.863772
0.01000,0.497006,0.744012,0.818862,0.826347,0.839820,0.850299,0.857784,0.868263,0.869760,0.866766,0.865269
0.03000,0.497006,0.727545,0.802395,0.824850,0.854790,0.859281,0.865269,0.874251,0.883234,0.875749,0.853293
0.10000,0.497006,0.497006,0.622754,0.755988,0.811377,0.844311,0.874251,0.869760,0.862275,0.857784,0.857784


### Se entrena el mejor modelo con los parámetros registrados y se obtienen las métricas más relevantes

In [26]:
final_model = SVC(C=10, 
                  kernel='rbf',
                  gamma=0.03)
final_model.fit(X,y)
score = final_model.score(X_test,y_test)

In [27]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

y_hat = final_model.predict(X_test)
f1 = f1_score(y_test, y_hat)
precision = precision_score(y_test, y_hat)
recall = recall_score(y_test, y_hat)
confusion = confusion_matrix(y_test, y_hat)

### Con el objetivo de observar con mayor claridad los resultados obtenidos, se genera un tabla con los valores para los descriptores encontrados para el modelo. Al igual que en casos anteriores, esta estimaciones poseen un 3.4% de presición con un 95.79% de confianza.

In [33]:
model_results = pd.DataFrame({
    "Accuracy" : [score],
    "Precisión" : [precision],
    "Recall" : [recall],
    "F1" : [f1]   
})
 
model_results

,Accuracy,Precisión,Recall,F1
0,0.862874,0.845209,0.869785,0.857321


### Se imprime la matriz de confusión y, a partir de ella, es posible evidenciar que el modelo tiene mayor dificultad clasificando caciones de jazz. Además, es posible afirmar que los resultados obtenidos con la máquina de soporte vectorial son ligeramente mejores a los obtenidos con las redes neuronales exploradas en trabajos anteriores. Sin embargo, aún no representan una mejora significativa.

In [32]:
confusion

array([[753, 126],
       [103, 688]], dtype=int64)